In [1]:
import sys
sys.path.append("./modules")


In [2]:
# Import general modules
from nansat import Nansat, Domain, NSR
import os 

# Import temporal modules needed for testing plotting
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Import SAR forecasting modules
import config
import s1_preparation
import domains_preparation
import SAR1_SAR2_drift_retrivial
import warping_with_domain

# Import variables
from config import path_to_HH_files, path_to_HV_files, safe_folder 
from config import output_folder, input_folder
from config import S1_prod_regex, S1_safe_regex
from config import lon, lat, X, Y, proj4, srs

# For cleaning up memory
import gc

import time

In [3]:
# 1. Prepare SAR pairs

# Collect Sentinel SAFE objects for files in safe directory.
safe_objects = s1_preparation.collect_sentinel_files(safe_folder, path_to_HH_files, path_to_HV_files,  S1_safe_regex, S1_prod_regex)

# Get pairs of Sentinel SAFE objects where their timestamps are within 50 hours of each other.
sar_pairs = s1_preparation.get_pairs_within_time_limit(safe_objects, hours = 50)

# Print details for each pair.
for index, pair in enumerate(sar_pairs, start=1):  # start=1 makes the index start from 1
    print(f'Pair {index}:')
    print(f'SAR1: {pair[0].filename} \ntimestamp: {pair[0].timestamp}\n'
          f'SAR2: {pair[1].filename} \ntimestamp: {pair[1].timestamp}')

Pair 1:
SAR1: S1A_EW_GRDM_1SDH_20230327T075355_20230327T075459_047827_05BF0B_E30B.SAFE 
timestamp: 2023-03-27 07:53:55
SAR2: S1A_EW_GRDM_1SDH_20230329T073729_20230329T073833_047856_05C011_8100.SAFE 
timestamp: 2023-03-29 07:37:29


In [4]:
import cProfile
import pstats
from io import StringIO






# Loop over all pairs and use enumerate to get an index for each pair
for index, pair in enumerate(sar_pairs, start=1):  # start=1 to have human-friendly indexing
    
    # Create a Profile object
    pr = cProfile.Profile()
    pr.enable()  # Start profiling
    
    start_time = time.time()
    
    # 2.1. Prepare nansat objects and domains for HV polarisation
    n1_hv, n2_hv, output_dir_name, plots_dir_hv = domains_preparation.prepare_nansat_objects(
        pair[0], pair[1], output_folder, polarisation='HV')
    
    # Prepare nansat objects and domains for HH polarisation
    n1_hh, n2_hh, output_dir_name, plots_dir_hh = domains_preparation.prepare_nansat_objects(
        pair[0], pair[1], output_folder, polarisation='HH')
    

    # Additional processing steps
    # 2.2  Define model domain (mod_dom) for comparing drift and comparison (dst_dom) domain to compare SAR images (real and forecasted)
    
    # Prepare subset model grid for domains and pattern matching
    X_subset, Y_subset, lon_subset, lat_subset = domains_preparation.prepare_grid(n1_hv, n2_hv, srs, X, Y, lon, lat, buffer=0)
    
    # Set a model domain
    mod_res = 2500
    mod_dom = Domain(srs, f'-te {min(X_subset.data)} {min(Y_subset.data) - mod_res * 2} {max(X_subset.data) + mod_res} {max(Y_subset.data)} -tr {mod_res} {mod_res}')
    
    
    lon1pm, lat1pm = mod_dom.get_geolocation_grids()
    x, y = mod_dom.get_geolocation_grids(dst_srs=srs)
    
    # Set a comparison domain 
    dst_res = 100
    dst_dom = Domain(srs, f'-te {min(X_subset.data)} {min(Y_subset.data) - dst_res * 2} {max(X_subset.data) + dst_res} {max(Y_subset.data)} -tr {dst_res} {dst_res}')
    
    domains_preparation.plot_borders(mod_dom, n1_hv, n2_hv, output_dir_name) # borders for hh and hv are the same
    # Checking that domains have the same borders 
    
    rows1, cols1 = dst_dom.shape()
    print("dst_dom corner coordinates:", dst_dom.transform_points([0,cols1-1,0,cols1-1], [0,0,rows1-1,rows1-1], dst_srs=srs))
    
    rows1, cols1 = mod_dom.shape()
    print("mod_dom corner coordinates:", mod_dom.transform_points([0,cols1-1,0,cols1-1], [0,0,rows1-1,rows1-1], dst_srs=srs))
    # 3.   Retrieve reference drift
    # 3.1. Run feature tracking and pattern matching for HV
    
   
    # Run feature tracking and plot results 
    c1_hv, r1_hv, c2_hv, r2_hv = SAR1_SAR2_drift_retrivial.run_feature_tracking(n1_hv, n2_hv, plots_dir_hv)
    
    #Run pattern matching and plot results
    upm_hv, vpm_hv, apm_hv, rpm_hv, hpm_hv, ssim_hv, lon2pm_hv, lat2pm_hv = SAR1_SAR2_drift_retrivial.run_pattern_matching(plots_dir_hv, x, y, 
                                                               lon1pm, lat1pm, n1_hv, c1_hv, r1_hv, n2_hv, c2_hv, r2_hv, srs, 
                                                               min_border=200,
                                                               max_border=200,
                                                               #min_border=10, #test
                                                               #max_border=10, #test
                                                               #angles=[-9,-6, -3, 0, 3, 6, 9]) #test
                                                               angles=[-50, -45, -40, -35, -30, -25, -20, -15,-12, -9,-6, -3, 0, 3, 6, 9, 12,15, 20, 25, 30, 35, 40, 45, 50])
    # 3.2. Run feature tracking and pattern matching for HH
    
    # HH Processing
    # Run feature tracking and plot results 
    c1_hh, r1_hh, c2_hh, r2_hh = SAR1_SAR2_drift_retrivial.run_feature_tracking(n1_hh, n2_hh, plots_dir_hh)
    
    #Run pattern matching and plot results
    upm_hh, vpm_hh, apm_hh, rpm_hh, hpm_hh, ssim_hh, lon2pm_hh, lat2pm_hh = SAR1_SAR2_drift_retrivial.run_pattern_matching(plots_dir_hh, x, y, 
                                                               lon1pm, lat1pm, n1_hh, c1_hh, r1_hh, n2_hh, c2_hh, r2_hh,srs, 
                                                               min_border=200,
                                                               max_border=200,
                                                               #min_border=10, #test
                                                               #max_border=10, #test
                                                               #angles=[-9,-6, -3, 0, 3, 6, 9]) #test
                                                               angles=[-50, -40, -35, -30, -25, -20, -15,-12, -9,-6, -3, 0, 3, 6, 9, 12,15, 20, 25, 30, 35, 40, 50 ])
    
    
    # 3.3. Get combined drift and all textural parameters
    
    # Combining hh and hv results based on hessian threshold
    upm, vpm, apm, rpm, hpm, ssim, lon2pm, lat2pm = SAR1_SAR2_drift_retrivial.combine_hh_hv(output_dir_name, x, y, upm_hh, vpm_hh, apm_hh, rpm_hh, hpm_hh, ssim_hh, lon2pm_hh, lat2pm_hh,
                                  upm_hv, vpm_hv, apm_hv, rpm_hv, hpm_hv, ssim_hv, lon2pm_hv, lat2pm_hv)
    # 3.4.  Get good pixel indices based on hessian and neighbor thresholds.
    
    #Returns:
    #    - gpi1: Good pixel index based on hessian value
    #    - gpi2: Good pixel index combining hessian and neighbors count 
    
    hessian=8
    neighbors=2
    
    gpi1, gpi2 = SAR1_SAR2_drift_retrivial.get_good_pixel_indices(hpm, h_threshold=hessian, neighbors_threshold=neighbors)
    
        
    # Plot the filtering results
    general_plots_path = SAR1_SAR2_drift_retrivial.plot_filter_results(output_dir_name, x, y, hpm, upm, vpm, gpi1, gpi2, hessian, neighbors)
    
    
    #  Save final drift, its parameters and filtering arrays to npy files
    save_name = 'sar_ref_drift_output'
    sar_drift_output_path = SAR1_SAR2_drift_retrivial.save_sar_drift_results(output_dir_name, save_name,
                                                                             upm=upm, vpm=vpm, apm=apm, rpm=rpm, 
                                                                             hpm=hpm, ssim=ssim, lon2pm=lon2pm, 
                                                                             lat2pm=lat2pm, gpi1=gpi1, gpi2=gpi2)
    # 4. Warp SAR1 image with the reference sar drift and compare all arrays in the comparison distination domain
    
    # 4.1. Warp
    # Warp SAR1 with SAR-drift compenstaion/displacement
    good_pixels = gpi2
    mask_pm = ~good_pixels # mask out low quality or NaN
    s1_dst_dom_S_hv = warping_with_domain.warp_with_uv(n1_hv, n1_hv[1], mod_dom, upm, vpm, mask_pm, dst_dom)
    s1_dst_dom_S_hh = warping_with_domain.warp_with_uv(n1_hh, n1_hh[1], mod_dom, upm, vpm, mask_pm, dst_dom)
    
    # Warp SAR2 to the comparison domain
    s2_dst_dom_hv = warping_with_domain.warp(n2_hv, n2_hv[1], dst_dom)
    s2_dst_dom_hh = warping_with_domain.warp(n2_hh, n2_hh[1], dst_dom)
    
    # Warp SAR1 to the comparison domain for visualisation
    s1_dst_dom_hv = warping_with_domain.warp(n1_hv, n1_hv[1], dst_dom)
    s1_dst_dom_hh = warping_with_domain.warp(n1_hh, n1_hh[1], dst_dom)
    # 4.2. Plot warping results
    warping_with_domain.plot_sar_forecast_images(general_plots_path, 
                                                 "Forecast_with_sar_ref_drift", 
                                                 s1_dst_dom_hv, s2_dst_dom_hv, s1_dst_dom_S_hv,
                                                 s1_dst_dom_hh, s2_dst_dom_hh, s1_dst_dom_S_hh,
                                                 gamma_value=1.2)
    # 5. Calculate quality parametrs (corr, hess, ssim) for the predicted SAR2 (by calculating pattern matchin on SAR2 and SAR2_predicted)
    
    # 5.1. Make new nansat objects for comparison
    
    n_s1_predict = Nansat.from_domain(dst_dom, array = s1_dst_dom_S_hv)
    n_s2 = Nansat.from_domain(dst_dom, array = s2_dst_dom_hv)
    
    # 5.2. Create directory for saving plots 
    comparison_dir = os.path.join(output_dir_name, f"comparison_plots")
    try:
        os.makedirs(comparison_dir, exist_ok=True)
        print(f"Successfully created {comparison_dir}")
    except Exception as e:
        print(f"Failed to create {comparison_dir}. Error: {e}")
        
    # Calculate realibility indexes 
    
    
    # 5.4. Run feature tracking and plot results 
    c1_alg_hv, r1_alg_hv, c2_alg_hv, r2_alg_hv = SAR1_SAR2_drift_retrivial.run_feature_tracking(n_s1_predict, n_s2, comparison_dir)
    
    # 5.5. Run pattern matching and plot results
    upm_alg_hv, vpm_alg_hv, apm_alg_hv, rpm_alg_hv, hpm_alg_hv, ssim_alg_hv, lon2pm_alg_hv, lat2pm_alg_hv = SAR1_SAR2_drift_retrivial.run_pattern_matching(comparison_dir, x, y, 
                                                               lon1pm, lat1pm, n_s1_predict, c1_alg_hv, r1_alg_hv, n_s2, c2_alg_hv, r2_alg_hv, srs, 
                                                               min_border=200,
                                                               max_border=200,
                                                               #min_border=10, #test
                                                               #max_border=10, #test
                                                               #angles=[-9,-6, -3, 0, 3, 6, 9]) #test
                                                               angles=[-50, -45, -40, -35, -30, -25, -20, -15,-12, -9,-6, -3, 0, 3, 6, 9, 12,15, 20, 25, 30, 35, 40, 45, 50])
    
    # 5.6. Save comparison results, its parameters and filtering arrays to npy files
    save_name = 'sar_drift_forecast_quality'
    sar_drift_output_path = SAR1_SAR2_drift_retrivial.save_sar_drift_results(comparison_dir, save_name,
                                                                             upm=upm, vpm=vpm, apm=apm, rpm=rpm, 
                                                                             hpm=hpm, ssim=ssim, lon2pm=lon2pm, 
                                                                             lat2pm=lat2pm, gpi1=gpi1, gpi2=gpi2)

    end_time = time.time()
    print(f"Pair {index} processed in {end_time - start_time:.2f} seconds.")
    
    pr.disable()  # Stop profiling
    s = StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
    ps.print_stats()

    # Get the profiling results as a string and print it
    profiling_results = s.getvalue()
    # Save the stats to a file
    profiling_dir_path = os.path.join(output_dir_name, "profiling")
    os.makedirs(profiling_dir_path, exist_ok=True)
    save_path = os.path.join(profiling_dir_path, f"profile_results_pair{index}.prof")
    print(f'profiling path is {save_path}')
    ps.dump_stats(save_path)

VMIN:  -4.618902969360351
VMAX:  6.8174938201904265
VMIN:  -4.023641300201416
VMAX:  6.247359943389878
Successfully created /home/jovyan/experiment_data/2022-2023_48h_experiment/batch_output/20230327T075355_20230329T073729
Successfully created /home/jovyan/experiment_data/2022-2023_48h_experiment/batch_output/20230327T075355_20230329T073729/HV_plots
VMIN:  -4.2762504577636715
VMAX:  5.969384288787829
VMIN:  -3.859353494644165
VMAX:  5.371604099273679
Successfully created /home/jovyan/experiment_data/2022-2023_48h_experiment/batch_output/20230327T075355_20230329T073729
Successfully created /home/jovyan/experiment_data/2022-2023_48h_experiment/batch_output/20230327T075355_20230329T073729/HH_plots
dst_dom corner coordinates: (array([278603.1875, 833603.1875, 278603.1875, 833603.1875]), array([662068.375, 662068.375, -32931.625, -32931.625]))
mod_dom corner coordinates: (array([278603.1875, 833603.1875, 278603.1875, 833603.1875]), array([662068.375, 662068.375, -32931.625, -32931.625]))
Ke

/home/jovyan/packages/2022-2023_48h_experiment/one_flow_package/./modules/sea_ice_drift/pmlib_with_ssim.py:60: RuntimeWarning: invalid value encountered in divide
  hes = (hes - np.median(hes)) / np.std(hes)


/home/jovyan/packages/2022-2023_48h_experiment/one_flow_package/./modules/sea_ice_drift/pmlib_with_ssim.py:60: RuntimeWarning: invalid value encountered in divide
  hes = (hes - np.median(hes)) / np.std(hes)


/home/jovyan/packages/2022-2023_48h_experiment/one_flow_package/./modules/sea_ice_drift/pmlib_with_ssim.py:60: RuntimeWarning: invalid value encountered in divide
  hes = (hes - np.median(hes)) / np.std(hes)


/home/jovyan/packages/2022-2023_48h_experiment/one_flow_package/./modules/sea_ice_drift/pmlib_with_ssim.py:60: RuntimeWarning: invalid value encountered in divide
  hes = (hes - np.median(hes)) / np.std(hes)


...

/home/jovyan/packages/2022-2023_48h_experiment/one_flow_package/./modules/sea_ice_drift/pmlib_with_ssim.py:60: RuntimeWarning: invalid value encountered in divide
  hes = (hes - np.median(hes)) / np.std(hes)


95% 02043.7 04401.7 03163.0 05063.0 -06.0 0.64 7.74 0.4668% 00159.2 02454.1 00959.0 03372.0 -09.0 0.11 1.77 0.1073% 01713.3 03313.2 02615.0 04043.0 +03.0 0.41 0.76 0.13
 Pattern matching - OK! ( 19 sec)
Arrays saved to /home/jovyan/experiment_data/2022-2023_48h_experiment/batch_output/20230327T075355_20230329T073729/sar_ref_drift_output/sar_ref_drift_output.npz
Successfully created /home/jovyan/experiment_data/2022-2023_48h_experiment/batch_output/20230327T075355_20230329T073729/comparison_plots
Key points found: 50000
Key points found: 50000
Domain filter: 50000 -> 50000
Domain filter: 50000 -> 50000
Keypoints matched 5.522753715515137
Ratio test 0.600000 found 2154 keypoints
MaxDrift filter: 2154 -> 2152
LSTSQ filter: 2152 -> 2152


/home/jovyan/packages/2022-2023_48h_experiment/one_flow_package/./modules/sea_ice_drift/pmlib_with_ssim.py:60: RuntimeWarning: invalid value encountered in divide
  hes = (hes - np.median(hes)) / np.std(hes)


/home/jovyan/packages/2022-2023_48h_experiment/one_flow_package/./modules/sea_ice_drift/pmlib_with_ssim.py:60: RuntimeWarning: invalid value encountered in divide
  hes = (hes - np.median(hes)) / np.std(hes)


80% 03525.0 05525.0 0000nan 0000nan +0nan 0nan 0nan 0nan058% 05000.0 04000.0 0000nan 0000nan +0nan 0nan 0nan 0nan53% 03675.0 03675.0 0000nan 0000nan +0nan 0nan 0nan 0nan80% 00550.0 05550.0 0000nan 0000nan +0nan 0nan 0nan 0nan80% 03050.0 05550.0 0000nan 0000nan +0nan 0nan 0nan 0nan55% 00825.0 03825.0 00815.0 03818.0 -06.0 -0.10 3.88 0.0782% 03175.0 05675.0 0000nan 0000nan +0nan 0nan 0nan 0nan95% 03575.0 06575.0 0000nan 0000nan +0nan 0nan 0nan 0nan76% 00775.0 05275.0 00773.0 05275.0 +00.0 0.73 10.61 0.5995% 00600.0 06600.0 0000nan 0000nan +0nan 0nan 0nan 0nan86% 02475.0 05975.0 0000nan 0000nan +0nan 0nan 0nan 0nan95% 03100.0 06600.0 0000nan 0000nan +0nan 0nan 0nan 0nan91% 00825.0 06325.0 0000nan 0000nan +0nan 0nan 0nan 0nan84% 03300.0 05800.0 0000nan 0000nan +0nan 0nan 0nan 0nan92% 00350.0 06350.0 0000nan 0000nan +0nan 0nan 0nan 0nan92% 05350.0 06350.0 0000nan 0000nan +0nan 0nan 0nan 0nan87% 01050.0 06050.0 0000nan 0000nan +0nan 0nan 0nan 0nan76% 03275.0 05275.0 0000nan 0000nan +0nan 0na

In [11]:
stats = pstats.Stats( "/home/jovyan/experiment_data/2022-2023_48h_experiment/batch_output/20230327T075355_20230329T073729/profiling/profile_results_pair1.prof")

stats.sort_stats("cumulative")
stats.print_stats(10)  # Print the top 10 functions

Wed Nov  8 16:19:51 2023    /home/jovyan/experiment_data/2022-2023_48h_experiment/batch_output/20230327T075355_20230329T073729/profiling/profile_results_pair1.prof

         24601869 function calls (23979703 primitive calls) in 293.601 seconds

   Ordered by: cumulative time
   List reduced from 3505 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.013    0.006  108.826   54.413 /home/jovyan/packages/2022-2023_48h_experiment/one_flow_package/./modules/domains_preparation.py:22(prepare_nansat_objects)
        4    1.020    0.255  100.853   25.213 /opt/conda/lib/python3.10/site-packages/sea_ice_drift-0.7.1-py3.10.egg/sea_ice_drift/lib.py:256(get_n)
        3    0.002    0.001   82.530   27.510 /home/jovyan/packages/2022-2023_48h_experiment/one_flow_package/./modules/SAR1_SAR2_drift_retrivial.py:155(run_pattern_matching)
       33    0.106    0.003   73.657    2.232 /opt/conda/lib/python3.10/site-packages/nansat/nansat.p

In [5]:
pip install snakeviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 979.3 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
!snakeviz /home/jovyan/experiment_data/2022-2023_48h_experiment/batch_output/20230327T075355_20230329T073729/profiling/profile_results_pair1.prof


snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Fhome%2Fjovyan%2Fexperiment_data%2F2022-2023_48h_experiment%2Fbatch_output%2F20230327T075355_20230329T073729%2Fprofiling%2Fprofile_results_pair1.prof
snakeviz: error: no web browser found: could not locate runnable browser

usage: snakeviz [-h] [-v] [-H ADDR] [-p PORT] [-b BROWSER_PATH] [-s] filename

Start SnakeViz to view a Python profile.

positional arguments:
  filename              Python profile to view

options:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit
  -H ADDR, --hostname ADDR
                        hostname to bind to (default: 127.0.0.1)
  -p PORT, --port PORT  port to bind to; if this port is already in use a free
                        port will be selected automatically (default: 8080)
  -b BROWSER_PATH, --browser BROWSER_PATH
                        name of webbrowser to launch as described i

In [ ]:
stats = pstats.Stats(profiling_file_path)